In [64]:
import csv
from datetime import datetime
from collections import defaultdict
inFile = "export.csv"
outFile = "bloodPressure.csv"
eveningBoundary = 15 # Exclusive, ie. <= eveningBoundary is morning
csvDelim = ","
arraySeparator = "; "
translations = {
    'date': 'Data',
    'morning': 'Rano',
    'evening': 'Wieczór'
}
emptyValueStr = ""

In [65]:
data = []
with open(inFile, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        data.append(row)

In [66]:
def formatNumber(num):
    # in my case, the blood pressure in the CSV was denoted like 120.000 for 120mmHg pressure, so I needed to cut 4 chars
    return num[:-4]

In [67]:
newData = defaultdict(dict)
for item in data[1:]:
    date = item[0]
    dt = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    dateStr= dt.strftime("%Y-%m-%d")
    key = "morning" if dt.hour <= 15 else "evening"
    value = f"{formatNumber(item[2])}/{formatNumber(item[1])}"
    if key not in newData[dateStr].keys():
        newData[dateStr][key] = [value]
    else: 
        newData[dateStr][key].append(value)
    

In [68]:
with open(outFile, 'w') as f:
    header = csvDelim.join(translations.values())
    f.write(f'{header}\n')
    for key in newData.keys():
        entry = newData[key]
        morning = arraySeparator.join(newData[key]["morning"]) if "morning" in newData[key] else emptyValueStr
        evening = arraySeparator.join(newData[key]["evening"]) if "evening" in newData[key] else emptyValueStr
        row = csvDelim.join([key, morning, evening])
        f.write(f'{row}\n')